In [6]:
import sys

sys.path.append("../")

import pandas as pd
from dotenv import load_dotenv

load_dotenv()
import os
import sqlite3

DB_PATH = os.getenv("DB_PATH")
DATA_PATH = "data"

conn = sqlite3.connect(DB_PATH)

if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

In [7]:
# Individuals Regions

df = pd.read_csv("../immaterial_index/results/df_individuals_score.csv")
print(len(set(df.individual_wikidata_id)))

df[df['individual_name']=='Hesiod']

159693


,Unnamed: 0,individual_wikidata_id,productive_year,decade,individual_name,score,region_code,region_name
52954,55510,Q44233,-715.0,-720,Hesiod,147,re_eastern_europe,Eastern Europe
52955,55511,Q44233,-715.0,-720,Hesiod,147,re_balkans,Balkans
52956,55512,Q44233,-715.0,-720,Hesiod,147,re_greek_world,Greek World
52957,55513,Q44233,-715.0,-720,Hesiod,147,re_mediterreanean,mediterranean World


In [8]:


# Load works of individuals

df_ind_works = pd.read_sql_query("SELECT * FROM individual_created_work", conn)

df_count_work = (
    df_ind_works.groupby("individual_wikidata_id")["work_wikidata_id"]
    .count()
    .rename("count_works")
    .reset_index()
)
df_final = pd.merge(df, df_count_work, on="individual_wikidata_id", how="left")
df_final = df_final.fillna(0)  # When there is no works we add 0
df_final = df_final.drop('Unnamed: 0', axis=1)
df_final = df_final.drop('productive_year', axis=1)
df_final.to_csv(DATA_PATH + "/df_indi_works.csv")


In [9]:
df_clean_gdp = pd.read_sql_query("SELECT * FROM gdp_clean", conn)
regions_clean = list(set(df_clean_gdp["region_code"]))
df = df[df["region_code"].isin(regions_clean)]
df = df[df["decade"] >= min(df_clean_gdp.year)]

df_final = pd.merge(df, df_count_work, on="individual_wikidata_id", how="left")
df_final = df_final.fillna(0)  # When there is no works we add 0
df_final.to_csv(DATA_PATH + "/df_indi_works_clean_gdp.csv")